# LangChain — Messages, ChatModels, Chains & LCEL
# -------------------------------------------------------------
# Make sure you have installed all the requirements


In [21]:
import os
from dotenv import load_dotenv
load_dotenv()

# MESSAGES
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# PROMPT TEMPLATES
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# CHAINING (OLD VERSION WITHOUT LCEL)
from langchain_classic.chains import LLMChain

# OUTPUT PARSER
from langchain_core.output_parsers.string import StrOutputParser

# CHAT MODELS
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_ollama import OllamaLLM
from langchain_mistralai import ChatMistralAI
from langchain_google_genai import GoogleGenerativeAI

# EMBEDDINGS
from langchain_huggingface import HuggingFaceEmbeddings


""" 
Many chat and embedding models are there in langchain.
some of them are opensource and use it for free.
And some of them are paid version, make sure to check the models availability and price before using it.

Don't forget to create API keys and store it in .env file before using it.
"""

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
api_key=os.getenv("GROQ_API_KEY")


python-dotenv could not parse statement starting at line 16


In [22]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [23]:

chat_ollama=OllamaLLM(model='llama3')

"""
Before using the ollama, it needs to be installed in the local system.
check it in the documentation for better understanding.
"""
chat_groq=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)

### Messages

In [24]:
# Common usage of messages with chat models
system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

messages = [system_msg, human_msg]
response = chat_groq.invoke(messages)  # It returns an AI_Message
response

AIMessage(content='Hello! I’m doing great—thanks for asking. How can I help you today?', additional_kwargs={'reasoning_content': 'User greeting. Need to respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 86, 'total_tokens': 122, 'completion_time': 0.035708273, 'prompt_time': 0.004084988, 'queue_time': 0.03685644, 'total_time': 0.039793261, 'completion_tokens_details': {'reasoning_tokens': 9}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e189667b30', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2d529362-8a0f-41fb-9ac0-34d726b407ed-0', usage_metadata={'input_tokens': 86, 'output_tokens': 36, 'total_tokens': 122})

In [29]:
# Message prompts: pass in a list of messages to the model by providing a list of message objects.
messages = [
    SystemMessage("You are a researcher writing excelent blogs on recent innovations"),
    HumanMessage("Write a blog post on AGI in 500 words"),
    AIMessage("AGI is known...")
]

response = chat_groq.invoke(messages)
response

AIMessage(content='**Artificial General Intelligence: The Dawn of a New Era**\n\nArtificial General Intelligence (AGI) – the idea that a machine could understand, learn, and apply knowledge across any domain, just like a human – has been a distant dream for decades. Yet, the past year has seen a surge of breakthroughs that bring us closer than ever to that horizon. In this post, we’ll unpack what AGI truly means, highlight the most exciting recent innovations, and examine the promises and perils that accompany this next frontier of technology.\n\n---\n\n### 1. From Narrow AI to General Intelligence\n\nFor years, AI systems have excelled at narrowly defined tasks: image classification, speech recognition, or playing Go. These “narrow” or “specialized” models thrive on massive labeled datasets and task-specific architectures. AGI, by contrast, would possess *flexible* reasoning, common sense, and the ability to transfer knowledge across contexts—an AI that can learn a new language, solve

In [30]:
# Dictionary format : specify messages directly in OpenAI chat completions format.
messages = [
    {"role": "system", "content": "You are a researcher writing excelent blogs on recent innovations"},
    {"role": "user", "content": "Write a blog post on AGI in 500 words"},
    {"role": "assistant", "content": "AGI is known..."}
]
response = chat_groq.invoke(messages)
response

AIMessage(content='**Artificial General Intelligence: The Dawn of Truly Adaptive Machines**\n\n*By Dr. Maya Patel – Researcher in Cognitive Systems, 2025*\n\n---\n\nArtificial General Intelligence (AGI) has long been the holy grail of AI research: a system that can understand, learn, and reason across any domain, just like a human. While narrow AI—think image classifiers, language models, and game‑playing agents—has become ubiquitous, AGI remains an elusive frontier. Yet, 2025 marks a turning point. In the past year alone, a series of breakthroughs have moved the field from theoretical speculation to tangible experimentation.\n\n### 1. From Scaling to Structure\n\nThe most celebrated AGI effort to date, *Unified Cognition Engine* (UCE), was unveiled by the OpenAI‑DeepMind consortium. Unlike the purely data‑driven models that dominated the 2020s, UCE integrates symbolic reasoning, probabilistic inference, and neural perception into a single architecture. Its “meta‑learning” core can rec

### Chains without LCEL

In [27]:

# A chain connects prompt + model + execution logic
summary_prompt = PromptTemplate.from_template("Summarize this paragraph:\n{text}")

llm_chain = LLMChain(
    prompt=summary_prompt,
    llm=chat_ollama
)

# Types of chains LLMChain, ConversationalChain, SequentialChain etc.

text_input = """
LangChain is the easiest way to start building agents and applications powered by LLMs.
With under 10 lines of code, you can connect to OpenAI, Anthropic, Google, and more.
LangChain provides a pre-built agent architecture and model integrations to help you 
get started quickly and seamlessly incorporate LLMs into your agents and applications.
We recommend you use LangChain if you want to quickly build agents and autonomous applications.
Use LangGraph, our low-level agent orchestration framework and runtime, when you have more
advanced needs that require a combination of deterministic and agentic workflows, heavy customization,
and carefully controlled latency.
LangChain agents are built on top of LangGraph in order to provide durable 
execution, streaming, human-in-the-loop, persistence, and more.
You do not need to know LangGraph for basic LangChain agent usage.
"""
result_chain = llm_chain.run({"text": text_input})
result_chain

"Here is a summary of the paragraph:\n\nLangChain is a tool that allows you to build agents and applications powered by Large Language Models (LLMs) with minimal code. It provides pre-built architecture and model integrations, making it easy to get started quickly. If you need more advanced features, LangGraph is recommended. LangChain agents are built on top of LangGraph and provide additional features like durable execution and streaming, but you don't need to know LangGraph for basic usage."

### LCEL — Modern, Pipe-based Composition
- Multiple chain types are there in LCEL using Runnables.


In [28]:

# LCEL allows chaining with '|' operator (Prompt → Model → Parser)
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to French: {sentence}"
)
parser = StrOutputParser()

lcel_chain = translate_prompt | chat_ollama | parser # This is how chaining happens using the pipe '|' operator. 
lcel_result = lcel_chain.invoke({"sentence": "Large Language Models are changing AI development."})
lcel_result


'Here is the translation:\n\nLes grands modèles de langage sont en train de révolutionner le développement de l\'IA.\n\nNote: "Large Language Models" can also be translated as "Grands modèles linguistiques" or "Grands modèles de langages", but "Les grands modèles de langage" is a more common and natural translation in French.'

- Messages — help structure role-based context.
- ChatModels — enable conversational LLMs with message awareness.
- Chains (without LCEL) — explicit setup, more control, more boilerplate.
- LCEL — concise, readable, and easier to debug.